In [1]:
import pandas as pd
import numpy as np

In [2]:
drinks = pd.read_csv('drinks.csv')

In [3]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


#### Loc/iloc

In [ ]:
# Select columns by name
drinks[['beer_servings','continent']]

In [ ]:
# Select columns with .loc. All rows for Continent
drinks.loc[:,'continent']

In [ ]:
# same but two columns and all rows
drinks.loc[:,['continent','beer_servings']]

In [ ]:
# everything for country Angola
drinks.loc[drinks['country']=='Angola',:]

In [ ]:
# only wine servings in Angola
drinks.loc[drinks['country']=='Angola', ['country','wine_servings']]

In [ ]:
# all countries where wine_servings > 45
drinks['country'].loc[drinks['wine_servings']>145]

In [ ]:
# all countries and continents, and servings where wine_servings > 145
drinks[['country','continent','wine_servings']].loc[drinks['wine_servings']>145]

In [ ]:
#select all even rows
drinks.iloc[::2,:]

In [ ]:
# select all odd columns
drinks.iloc[:,::2]

In [ ]:
# select the last column without using the name
drinks.iloc[:,-1:]

In [ ]:
# select all but the last column without using the name
drinks.iloc[:,:-1]

In [ ]:
# Rename a column in place
# drinks = drinks.rename({'country':'Country'}, axis = 0, inplace = True)

In [ ]:
drinks.rename(columns={"country":"Country"}, inplace = True)

In [ ]:
drinks.columns

In [ ]:
# Selecting everything for a given list of countries
drinks[drinks['Country'].isin(['Algeria','Spain','Canada'])]

In [ ]:
# The opposite of the above: all countries except the given three
drinks[~drinks['Country'].isin(['Algeria','Spain','Canada'])]

In [ ]:
# select columns by data type
drinks.dtypes

In [ ]:
# select first 15 rows
drinks.iloc[:15,:]

In [ ]:
#let's select float64
drinks.select_dtypes(include='float64')

In [ ]:
# let's exclude objects
drinks.select_dtypes(exclude = 'object')

In [ ]:
# For continent: change its type object to type category (saves space)
dtypes ={'continent':'category'}
drinks.astype(dtype = dtypes).dtypes

In [29]:
# Select top3 countries by beer_servings
drinks.nlargest(3,['beer_servings'])

,Country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
117,Namibia,376,3,1,6.8,Africa
45,Czech Republic,361,170,134,11.8,Europe
62,Gabon,347,98,59,8.9,Africa


In [34]:
# Same thing as above but in a more mundane way: pass the index of rows with top3 countries to iloc
drinks.iloc[drinks['beer_servings'].nlargest(3).index,:]

,Country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
117,Namibia,376,3,1,6.8,Africa
45,Czech Republic,361,170,134,11.8,Europe
62,Gabon,347,98,59,8.9,Africa


In [ ]:
# dealing with nans, % of nans in a column, split a string, pivot tables, correlation heatmap, aggregate functions (agg, mean(), group by)
# https://www.youtube.com/watch?v=RlIiVeig3hc
# https://www.youtube.com/watch?v=dPwLlJkSHLo

#### NaNs

In [35]:
# how many missing values are in every column
drinks.isna().sum()

Country                         0
beer_servings                   0
spirit_servings                 0
wine_servings                   0
total_litres_of_pure_alcohol    0
continent                       0
dtype: int64

In [36]:
# % of missing values
drinks.isna().mean()
# alternative
drinks.isna().sum() / len(drinks)

Country                         0.0
beer_servings                   0.0
spirit_servings                 0.0
wine_servings                   0.0
total_litres_of_pure_alcohol    0.0
continent                       0.0
dtype: float64

In [ ]:
# drop columns where more than 10% of values are missing
drinks.dropna(thresh=len(drinks)*0.9, axis = 'columns')

In [ ]:
# fill NaNs with something
drinks.fillna(0) # this case it will be zero
drinks.fillna(method='bfill') # fill with the next value 
drinks.interpolate(method='linear', limit_direction='forward') # interpolate 
drinks.dropna(axis = 1) # drop columns with NaNs

#### Splitting cells

In [39]:
names = pd.DataFrame({'name':['John Arthur Doe', 'Jane Ann Smith'],
                   'location':['Los Angeles, CA', 'Washington, DC']})

In [40]:
names.head()

,name,location
0,John Arthur Doe,"Los Angeles, CA"
1,Jane Ann Smith,"Washington, DC"


In [41]:
# split the column name into three columns and add these columns to the dataset
names[['first','middle','last']]= names['name'].str.split(' ', expand = True)

In [43]:
# split location into two columns but add only one resulting column to the dataset
names['State'] = names['location'].str.split(', ', expand = True)[1]

#### Aggregate functions

In [4]:
# First we need to load a new dataset
chipotle = pd.read_csv('chipotle.csv', error_bad_lines=False, header = None, sep='\\')

In [5]:
chipotle = chipotle.iloc[1:,:]

In [6]:
chipotle.columns.values

array([0])

In [7]:
chipotle[['order_id','quantity','item_name','choice_description','item_price']] = chipotle.iloc[:,0].str.split('\t', expand = True)

In [8]:
# Drop the first column
chipotle.drop([0],axis = 'columns', inplace = True)

In [9]:
chipotle.head()
# order_id	quantity	item_name	choice_description	item_price

,order_id,quantity,item_name,choice_description,item_price
1,1,1,Chips and Fresh Tomato Salsa,NULL,$2.39
2,1,1,Izze,[Clementine],$3.39
3,1,1,Nantucket Nectar,[Apple],$3.39
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,$2.39
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [10]:
chipotle.dtypes

order_id              object
quantity              object
item_name             object
choice_description    object
item_price            object
dtype: object

In [11]:
# remove the $ sign from item_price in order to convert it to float32 later
chipotle['item_price'] = chipotle['item_price'].str[1:] 

In [12]:
chip_dtypes ={'quantity':'int','item_price': np.float32}
chipotle = chipotle.astype(dtype = chip_dtypes)

In [13]:
chipotle.dtypes

order_id               object
quantity                int64
item_name              object
choice_description     object
item_price            float32
dtype: object

In [14]:
# now we start aggregating
# Calculate the total price of the order: sum item_price for a given order_id
chipotle.groupby('order_id')['item_price'].sum().head()

order_id
1       11.56
10      13.20
100     10.08
1000    20.50
1001    10.08
Name: item_price, dtype: float32

In [15]:
# passing a list of functions to groupby
chipotle.groupby('order_id')['item_price'].agg(['sum','count','std']).head()

,sum,count,std
order_id,,,
1,11.56,4,0.577350
10,13.20,2,3.040559
100,10.08,2,5.586143
1000,20.50,2,1.414214
1001,10.08,2,5.586143


In [16]:
# create a new column showing the total price of each order. Values will repeat b/c the table is on the item level
chipotle['Total Price'] = chipotle.groupby('order_id')['item_price'].transform('sum')

In [17]:
chipotle.head()

,order_id,quantity,item_name,choice_description,item_price,Total Price
1,1,1,Chips and Fresh Tomato Salsa,NULL,2.39,11.56
2,1,1,Izze,[Clementine],3.39,11.56
3,1,1,Nantucket Nectar,[Apple],3.39,11.56
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,2.39,11.56
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,16.98


In [18]:
# calculate the % of item price in the order price: divide one column by another
chipotle['Percentage of order price'] = chipotle['item_price']/chipotle['Total Price']

In [19]:
chipotle.head()

,order_id,quantity,item_name,choice_description,item_price,Total Price,Percentage of order price
1,1,1,Chips and Fresh Tomato Salsa,NULL,2.39,11.56,0.206747
2,1,1,Izze,[Clementine],3.39,11.56,0.293253
3,1,1,Nantucket Nectar,[Apple],3.39,11.56,0.293253
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,2.39,11.56,0.206747
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,16.98,1.000000


#### Pivot tables

In [20]:
# On the Titanic dataset
titanic_train = pd.read_csv('titanic_train.csv')

In [21]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
# straightforward pivot table
titanic_train.pivot_table(index='Sex', columns = 'Pclass', values = 'Survived', aggfunc = 'count', margins = True)

Pclass,1,2,3,All
Sex,,,,
female,94,76,144,314
male,122,108,347,577
All,216,184,491,891


In [23]:
# create bins and add as a column to the dataframe 
titanic_train['Age category'] = pd.cut(titanic_train['Age'], bins = [0,18,25,99], labels = ['child','young adult','adult']).head()

In [24]:
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age category
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,young adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,adult


#### Running total

In [43]:
# Something very simple to start with
ids = [[1, 100], [2, 200], [3, 300]]
ids_df = pd.DataFrame(ids, columns = ['id','Num'])

In [44]:
ids_df

,id,Num
0,1,100
1,2,200
2,3,300


In [46]:
ids_df['Cum_total'] = ids_df['Num'].cumsum()

In [47]:
ids_df

,id,Num,Cum_total
0,1,100,100
1,2,200,300
2,3,300,600


In [25]:
# If we want to calculate a running total value by group

students = [['Sally','Female', 14], ['Edward', 'Male', 12], ['Jon','Male',13], ['Liana','Female', 10],
           ['Ben', 'Male', 11], ['Elice', 'Female', 12], ['Nick', 'Male', 12], ['Josh','Male',12], 
           ['Lisa','Female', 10], ['Wick', 'Male', 15]]
students_df = pd.DataFrame(students, columns = ['Name', 'Gender', 'Age'])

In [26]:
students_df

,Name,Gender,Age
0,Sally,Female,14
1,Edward,Male,12
2,Jon,Male,13
3,Liana,Female,10
4,Ben,Male,11
5,Elice,Female,12
6,Nick,Male,12
7,Josh,Male,12
8,Lisa,Female,10
9,Wick,Male,15


In [39]:
# Let's calculate the running total by gender
students_df.groupby(by=['Gender','Name'])['Age'].sum().groupby(level=[0]).cumsum()

#We group by Gender and Name and calculate the cumulative sum of Age within the given Gender (it's what level=[0] does)

Gender  Name  
Female  Elice     12
        Liana     22
        Lisa      32
        Sally     46
Male    Ben       11
        Edward    23
        Jon       36
        Josh      48
        Nick      60
        Wick      75
Name: Age, dtype: int64

#### Lag function

In [100]:
# Imagine you have a daily sales information and want to calculate how prices change on a daily basis
selling = [ [1, 3400, '2020-01-01', 52],[1, 3111, '2020-01-01', 110],[2, 1967, '2020-01-02', 351],
            [2, 3400, '2020-01-02', 9], [3, 3400, '2020-01-02', 14], [3, 3111, '2020-01-02', 66],
            [3, 1405, '2020-01-02', 103]]
selling_df = pd.DataFrame(selling, columns = ['seller_id','product_id','sale_date','sales_sum'])


In [101]:
cat_types = {'seller_id':'category','product_id':'category'}
selling_df.astype(dtype=cat_types).dtypes

seller_id     category
product_id    category
sale_date       object
sales_sum        int64
dtype: object

In [102]:
selling_df['sale_date'] = selling_df['sale_date'].astype('datetime64')

In [103]:
selling_df.dtypes

seller_id              int64
product_id             int64
sale_date     datetime64[ns]
sales_sum              int64
dtype: object

In [104]:
selling_df

,seller_id,product_id,sale_date,sales_sum
0,1,3400,2020-01-01,52
1,1,3111,2020-01-01,110
2,2,1967,2020-01-02,351
3,2,3400,2020-01-02,9
4,3,3400,2020-01-02,14
5,3,3111,2020-01-02,66
6,3,1405,2020-01-02,103


In [105]:
# First, we rolled up everything to the daily level
selling_df_daily = selling_df.groupby(by='sale_date')['sales_sum'].sum().reset_index()

In [106]:
selling_df_daily

,sale_date,sales_sum
0,2020-01-01,162
1,2020-01-02,543


In [107]:
# Second, we will add a shifted column
selling_df_daily['prev_sales_sum'] = selling_df_daily['sales_sum'].shift(1)

In [108]:
selling_df_daily

,sale_date,sales_sum,prev_sales_sum
0,2020-01-01,162,NaN
1,2020-01-02,543,162.0


In [109]:
# Third, we will calculate the daily difference 
selling_df_daily['daily_diff'] = selling_df_daily['sales_sum'] - selling_df_daily['prev_sales_sum']

In [110]:
selling_df_daily

,sale_date,sales_sum,prev_sales_sum,daily_diff
0,2020-01-01,162,NaN,NaN
1,2020-01-02,543,162.0,381.0


#### SQL-like exercises

In [118]:
add4add = [
    ['2018-03-15','impression',2353, 3436,0.15,np.nan,np.nan],
    ['2018-03-15','impression',2353, 3436,0.12,np.nan,np.nan],
    ['2018-03-15','impression',2353, 3436, 0.10,np.nan,np.nan],
    ['2018-03-15','click',1234,5678, np.nan, np.nan,np.nan],
    ['2018-03-15','create_ad', 1234, 5678 , np.nan, 6789,10],
    ['2018-03-16','impression',8765,8871,0.10,np.nan,np.nan],
    ['2018-03-16','impression',8765,8871,0.10,np.nan,np.nan],
    ['2018-03-16','impression',8765, 8871,0.2,np.nan,np.nan],
    ['2018-03-16','impression',8765,8871,0.23,np.nan,np.nan],
    ['2018-03-16','click',8765,8871, np.nan, np.nan,np.nan],
    ['2018-03-16','create_ad','8765',8871, np.nan, 6993,12] ]

add4add_df = pd.DataFrame(add4add, columns = ['event_date','event','user_id','unit_id','cost','ad_id','spend'])

In [119]:
add4add_df.dtypes

event_date     object
event          object
user_id        object
unit_id         int64
cost          float64
ad_id         float64
spend         float64
dtype: object

In [123]:
add4add_df['event_date'] = add4add_df['event_date'].astype('datetime64')
cat_types = {'event':'category','user_id':'category', 'unit_id':'category'}
add4add_df.astype(dtype=cat_types)

,event_date,event,user_id,unit_id,cost,ad_id,spend
0,2018-03-15,impression,2353,3436,0.15,NaN,NaN
1,2018-03-15,impression,2353,3436,0.12,NaN,NaN
2,2018-03-15,impression,2353,3436,0.10,NaN,NaN
3,2018-03-15,click,1234,5678,NaN,NaN,NaN
4,2018-03-15,create_ad,1234,5678,NaN,6789.0,10.0
5,2018-03-16,impression,8765,8871,0.10,NaN,NaN
6,2018-03-16,impression,8765,8871,0.10,NaN,NaN
7,2018-03-16,impression,8765,8871,0.20,NaN,NaN
8,2018-03-16,impression,8765,8871,0.23,NaN,NaN
9,2018-03-16,click,8765,8871,NaN,NaN,NaN


In [ ]:
# add the country table next and do the first simple question 

In [ ]:
# The question is to count the average number of impressions we need to show before a user will create an add
# In other words, we need to count 'impression' for every user_id and unit_id where we have 'create_ad' at the end
